In [217]:
import pandas as pd

from collections import UserDict, UserList, Counter
from tqdm import tqdm
from cached_property import cached_property
from itertools import islice, chain
from functools import lru_cache
from torch.utils.data import random_split

from news_vec.utils import read_json_gz_lines
from news_vec import logger

In [40]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import altair as alt
import seaborn as sns

mpl.style.use('seaborn-muted')
sns.set(style="whitegrid")

%matplotlib inline

In [213]:
class Corpus:

    def __init__(self, headline_root, skim=None):
        """Read headline df.
        """
        logger.info('Reading headlines.')
        
        lines = islice(read_json_gz_lines(headline_root), skim)
        self.df = pd.DataFrame(list(tqdm(lines)))

    def __repr__(self):

        pattern = '{cls_name}<{hl_count} headlines>'

        return pattern.format(
            cls_name=self.__class__.__name__,
            hl_count=len(self.df),
        )
    
    @cached_property
    def min_db_count(self):
        return self.df.groupby(['domain', 'ts_bucket']).size().min()
    
    def sample_all_vs_all(self):
        return self.df.groupby(['domain', 'ts_bucket']).apply(lambda x: x.sample(self.min_db_count))
    
    @lru_cache(None)
    def filter_ab(self, d1, d2):
        return self.df[self.df.domain.isin([d1, d2])].groupby(['domain', 'ts_bucket'])
    
    def sample_ab(self, d1, d2):
        return self.filter_ab(d1, d2).apply(lambda x: x.sample(self.min_db_count))
    
    @lru_cache(None)
    def filter_ab_ts(self, d1, d2, bucket):
        return self.df[self.df.domain.isin([d1, d2])&(self.df.ts_bucket==bucket)].groupby('domain')
    
    def sample_ab_ts(self, d1, d2, bucket):
        return self.filter_ab_ts(d1, d2, bucket).apply(lambda x: x.sample(self.min_db_count))

In [226]:
class HeadlineDataset(UserList):
    
    @classmethod
    def from_df(cls, df, label_col='domain', **kwargs):
        pairs = [(d, d[label_col]) for d in df.to_dict('records')]
        return cls(pairs, **kwargs)

    def __init__(self, pairs, test_frac=0.1):
        """Set train/val/test splits.
        """
        test_size = round(len(pairs) * test_frac)
        train_size = len(pairs) - (test_size * 2)

        sizes = (train_size, test_size, test_size)
        self.train, self.val, self.test = random_split(pairs, sizes)

    def __iter__(self):
        return chain(self.train, self.val, self.test)

    def __repr__(self):

        pattern = '{cls_name}<{train_size}/{val_size}/{test_size}>'

        return pattern.format(
            cls_name=self.__class__.__name__,
            train_size=len(self.train),
            val_size=len(self.val),
            test_size=len(self.test),
        )

In [224]:
c = Corpus('../data/clf-headlines.json/', None)

2018-12-27 12:20:11,566 | INFO : Reading headlines.
1225511it [00:44, 27273.37it/s] 


In [227]:
c.min_db_count

2686

In [234]:
ds = HeadlineDataset.from_df(c.sample_ab_ts('nytimes.com', 'apnews.com', 0))

In [235]:
ds

HeadlineDataset<4298/537/537>

In [239]:
%time c.sample_ab_ts('nytimes.com', 'apnews.com', 0).head(10)

CPU times: user 19.2 ms, sys: 1.83 ms, total: 21 ms
Wall time: 19.4 ms


article_id  \
domain                              
apnews.com 932722    601295464600   
           1125824  1365799605635   
           1021123  1236950591462   
           673189     77309416169   
           202236    283467877822   
           326673    867583417162   
           1120554  1614907722435   
           213621     60129561728   
           297547   1271310335695   
           18645     111669154341   

                                                           clf_tokens  \
domain                                                                  
apnews.com 932722   [airstrikes, killed, #, is, group, linked, mil...   
           1125824  [somaliland, asks, us, for, exemption, to, tru...   
           1021123     [the, most, unique, one, day, job, in, sports]   
           673189   [baby, chimp, abandoned, by, mom, to, join, ka...   
           202236   [de, sacerdotes, australianos, acusados, de, a...   
           326673   [chris, brown, to, box, soulja, boy, over, soc...   
           1120554  [travis, snider, agrees, to, minor, league, de...   
           213621   [torres, taken, to, hospital, after, head, inj...   
           297547   [japan, pm, pledges, unchanging, allegiance, t...   
           18645    [new, leader, orders, justice, to, do, our, sw...   

                        domain  impressions  \
domain                                        
apnews.com 932722   apnews.com        12257   
           1125824  apnews.com        37083   
           1021123  apnews.com       116313   
           673189   apnews.com        51349   
           202236   apnews.com        77516   
           326673   apnews.com        13605   
           1120554  apnews.com       108875   
           213621   apnews.com        12135   
           297547   apnews.com       207968   
           18645    apnews.com        36419   

                                                               tokens  \
domain                                                                  
apnews.com 932722   [Philippines, :, Airstrikes, killed, 15, IS, g...   
           1125824  [Somaliland, asks, US, for, exemption, to, Tru...   
           1021123  [Emergency, goalie, :, The, most, unique, one,...   
           673189   [Baby, chimp, abandoned, by, mom, to, join, Ka...   
           202236   [7, %, de, sacerdotes, australianos, ,, acusad...   
           326673   [Chris, Brown, to, box, Soulja, Boy, over, soc...   
           1120554  [Travis, Snider, agrees, to, minor, league, de...   
           213621   [Torres, taken, to, hospital, after, head, inj...   
           297547   [Japan, PM, pledges, ', unchanging, ', allegia...   
           18645    [The, Latest, :, New, leader, orders, Justice,...   

                    ts_bucket  
domain                         
apnews.com 932722           0  
           1125824          0  
           1021123          0  
           673189           0  
           202236           0  
           326673           0  
           1120554          0  
           213621           0  
           297547           0  
           18645            0

In [241]:
%time c.sample_ab('nytimes.com', 'apnews.com').head(10)

CPU times: user 170 ms, sys: 8.25 ms, total: 178 ms
Wall time: 177 ms


article_id  \
domain     ts_bucket                          
apnews.com 0         1153273   343597392855   
                     320551    824633721167   
                     907780    472446440775   
                     440927    171798698465   
                     130997    670014922942   
                     603477   1133871395779   
                     1128338   335007495429   
                     197996    833223679361   
                     24626      51539645070   
                     1000069   549755816476   

                                                                     clf_tokens  \
domain     ts_bucket                                                              
apnews.com 0         1153273  [lyon, s, defense, folds, again, in, #, #, def...   
                     320551   [at, least, #, killed, in, ethnic, clashes, in...   
                     907780   [china, suspends, coal, imports, from, north, ...   
                     440927   [walker, s, state, of, state, to, include, com...   
                     130997   [senate, confirms, trump, s, nominee, for, us,...   
                     603477   [bonds, clemens, making, slow, gains, with, ch...   
                     1128338  [chimera, greiss, lead, islanders, to, #, #, w...   
                     197996                           [the, audacity, of, hype]   
                     24626    [protests, erupt, after, kyrgyzstan, arrests, ...   
                     1000069  [senators, want, materials, saved, for, russia...   

                                  domain  impressions  \
domain     ts_bucket                                    
apnews.com 0         1153273  apnews.com       109828   
                     320551   apnews.com        23807   
                     907780   apnews.com        21711   
                     440927   apnews.com        23575   
                     130997   apnews.com       168999   
                     603477   apnews.com        30405   
                     1128338  apnews.com       109803   
                     197996   apnews.com        11204   
                     24626    apnews.com        46876   
                     1000069  apnews.com        26212   

                                                                         tokens  \
domain     ts_bucket                                                              
apnews.com 0         1153273  [Lyon, 's, defense, folds, again, in, 2, 1, de...   
                     320551   [Govt, :, At, least, 13, killed, in, ethnic, c...   
                     907780   [China, suspends, coal, imports, from, North, ...   
                     440927   [AP, News, :, Walker, 's, State, of, State, to...   
                     130997   [Senate, confirms, Trump, 's, nominee, for, US...   
                     603477   [Bonds, ,, Clemens, making, slow, gains, with,...   
                     1128338  [Chimera, ,, Greiss, lead, Islanders, to, 3, 1...   
                     197996       [AP, FACT, CHECK, :, The, audacity, of, hype]   
                     24626    [Protests, erupt, after, Kyrgyzstan, arrests, ...   
                     1000069  [AP, source, :, Senators, want, materials, sav...   

                              ts_bucket  
domain     ts_bucket                     
apnews.com 0         1153273          0  
                     320551           0  
                     907780           0  
                     440927           0  
                     130997           0  
                     603477           0  
                     1128338          0  
                     197996           0  
                     24626            0  
                     1000069          0